In [1]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy
import pandas as pd
from sodapy import Socrata
from sqlalchemy import create_engine



# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us","H1zifl7IkGjHmDxwbQMS6lRPs", 
                 username = "lmaldonado93@gmail.com",
                 password = "gJVmWBgNzS52Vwy")

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 700000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("8h9b-rp9u", limit = 700000, order="arrest_date desc")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.head()

,age_group,arrest_boro,arrest_date,arrest_key,arrest_precinct,jurisdiction_code,ky_cd,latitude,law_cat_cd,law_code,longitude,ofns_desc,pd_cd,pd_desc,perp_race,perp_sex,x_coord_cd,y_coord_cd
0,45-64,K,2018-12-31T00:00:00.000,191703953,81,0,347,40.677026245000036,M,VTL11920U2,-73.92181884199994,INTOXICATED & IMPAIRED DRIVING,905,"INTOXICATED DRIVING,ALCOHOL",BLACK,M,1005936,185942
1,45-64,Q,2018-12-31T00:00:00.000,191692043,100,0,361,40.586231405000035,M,PL 2403002,-73.81601112099996,OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV,639,AGGRAVATED HARASSMENT 2,WHITE,F,1035354,152907
2,18-24,M,2018-12-31T00:00:00.000,191703279,14,0,113,40.75818729600008,F,PL 1701003,-73.98800312899994,FORGERY,729,"FORGERY,ETC.,UNCLASSIFIED-FELONY",BLACK,M,987574,215502
3,18-24,Q,2018-12-31T00:00:00.000,191703706,103,0,348,40.70059059000005,M,VTL05110A2,-73.80774319999993,VEHICLE AND TRAFFIC LAWS,922,"TRAFFIC,UNCLASSIFIED MISDEMEANOR",BLACK,F,1037559,194576
4,25-44,K,2018-12-31T00:00:00.000,191704095,67,0,341,40.662518015000046,M,PL 1552500,-73.92701180399997,PETIT LARCENY,339,"LARCENY,PETIT FROM OPEN AREAS,UNCLASSIFIED",BLACK,M,1004500,180655


In [2]:
# Create a filtered dataframe from specific columns
results_cols = ['arrest_key','arrest_date','arrest_precinct',
                'arrest_boro', 'perp_race', 
                'perp_sex', 'age_group',
                'ofns_desc']
results_transformed = results_df[results_cols].copy()

# Rename the column headers
results_transformed= results_transformed.rename(columns={'arrest_key':'id',
                                                      'arrest_precinct':'precinct',
                                                       'arrest_boro':'borough',
                                                       'ofns_desc':'offense'})
# Change arrest_date columns type to datetime64
results_transformed['arrest_date'] = pd.to_datetime(results_transformed.arrest_date)

# Clean the data by dropping duplicates and setting the index
results_transformed.drop_duplicates("id", inplace=True)
results_transformed.set_index("id", inplace=True)

results_transformed.head()



,arrest_date,precinct,borough,perp_race,perp_sex,age_group,offense
id,,,,,,,
191703953,2018-12-31,81,K,BLACK,M,45-64,INTOXICATED & IMPAIRED DRIVING
191692043,2018-12-31,100,Q,WHITE,F,45-64,OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV
191703279,2018-12-31,14,M,BLACK,M,18-24,FORGERY
191703706,2018-12-31,103,Q,BLACK,F,18-24,VEHICLE AND TRAFFIC LAWS
191704095,2018-12-31,67,K,BLACK,M,25-44,PETIT LARCENY


In [3]:
results_transformed.dtypes


arrest_date    datetime64[ns]
precinct               object
borough                object
perp_race              object
perp_sex               object
age_group              object
offense                object
dtype: object

In [4]:
results_transformed.arrest_date.dt.year.value_counts()

2017    286225
2018    246773
2016    167002
Name: arrest_date, dtype: int64

In [5]:
results_transformed.head()

,arrest_date,precinct,borough,perp_race,perp_sex,age_group,offense
id,,,,,,,
191703953,2018-12-31,81,K,BLACK,M,45-64,INTOXICATED & IMPAIRED DRIVING
191692043,2018-12-31,100,Q,WHITE,F,45-64,OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV
191703279,2018-12-31,14,M,BLACK,M,18-24,FORGERY
191703706,2018-12-31,103,Q,BLACK,F,18-24,VEHICLE AND TRAFFIC LAWS
191704095,2018-12-31,67,K,BLACK,M,25-44,PETIT LARCENY


In [6]:
results_transformed.perp_race.unique()

array(['BLACK', 'WHITE', 'ASIAN / PACIFIC ISLANDER', 'WHITE HISPANIC',
       'UNKNOWN', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'],
      dtype=object)

In [7]:
# Replace values in columns
result_replace_borough = results_transformed['borough'].replace({'K':'Brooklyn','Q':'Queens','M':'Manhattan','S':'Staten Island','B':"Bronx"},inplace=True)
result_replace_perp_sex = results_transformed['perp_sex'].replace({'M':'Male','F':'Female'},inplace=True)
result_replace_perp_race = results_transformed['perp_race'].replace({'BLACK':'Black','WHITE':'White','ASIAN / PACIFIC ISLANDER':'Asian / Pacific Islander','WHITE HISPANIC':'White Hispanic','UNKNOWN':"Unknown", 'BLACK HISPANIC':'Black Hispanic', 'AMERICAN INDIAN/ALASKAN NATIVE':'American Indian/Alaskan Native'},inplace=True)

In [8]:
results_transformed.head()

,arrest_date,precinct,borough,perp_race,perp_sex,age_group,offense
id,,,,,,,
191703953,2018-12-31,81,Brooklyn,Black,Male,45-64,INTOXICATED & IMPAIRED DRIVING
191692043,2018-12-31,100,Queens,White,Female,45-64,OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV
191703279,2018-12-31,14,Manhattan,Black,Male,18-24,FORGERY
191703706,2018-12-31,103,Queens,Black,Female,18-24,VEHICLE AND TRAFFIC LAWS
191704095,2018-12-31,67,Brooklyn,Black,Male,25-44,PETIT LARCENY


In [9]:
#filter dates from start of 2017 to end of 2018
results_1718 = results_transformed[(results_transformed["arrest_date"]> "2016-12-31") & (results_transformed["arrest_date"]< "2019-01-01")]
results_1718.to_csv("Resources/results_1718")

results_1718.head()

,arrest_date,precinct,borough,perp_race,perp_sex,age_group,offense
id,,,,,,,
191703953,2018-12-31,81,Brooklyn,Black,Male,45-64,INTOXICATED & IMPAIRED DRIVING
191692043,2018-12-31,100,Queens,White,Female,45-64,OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV
191703279,2018-12-31,14,Manhattan,Black,Male,18-24,FORGERY
191703706,2018-12-31,103,Queens,Black,Female,18-24,VEHICLE AND TRAFFIC LAWS
191704095,2018-12-31,67,Brooklyn,Black,Male,25-44,PETIT LARCENY


In [10]:
results_1718.arrest_date.dt.year.value_counts()

2017    286225
2018    246773
Name: arrest_date, dtype: int64

In [11]:
user_name = input('Enter user name:')
password = input('Password:')
rds_connection_string = "{0}:{1}@localhost:5432/nyc_arrest_db".format(user_name, password)
engine = create_engine(f'postgresql://{rds_connection_string}')

Enter user name:postgres
Password:Asuna0718


### -- Create Tables in POSTGRES SQL
CREATE TABLE arrest_1718 (
  id INT PRIMARY KEY,
  arrest_date DATE,
	precinct INT,
	borough text,
	perp_race text,
	perp_sex text,
	age_group text,
	offense text
);




In [12]:
engine.execute("CREATE TABLE IF NOT EXISTS arrest_1718 (id INT PRIMARY KEY, arrest_date DATE, precinct INT, borough text, perp_race text, perp_sex text, age_group text, offense text)")  

In [13]:
engine.table_names()

['arrest_1718']

In [14]:
results_1718.to_sql(name='arrest_1718', con=engine, if_exists='append', index=True)